In [144]:
import pandas as pd
import numpy as np

relevant_housing = [
    'period_begin', 'region', 'city', 'state', 
    'median_sale_price', 'homes_sold', 
    'median_list_price', 'median_dom'
]

# Import only the relevant columns
housing_data = pd.read_csv('neighborhood_market_tracker.tsv000', sep='\t', usecols=relevant_housing)

In [85]:
housing_data.info()
housing_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9002881 entries, 0 to 9002880
Data columns (total 8 columns):
 #   Column             Dtype  
---  ------             -----  
 0   period_begin       object 
 1   region             object 
 2   city               object 
 3   state              object 
 4   median_sale_price  float64
 5   median_list_price  float64
 6   homes_sold         float64
 7   median_dom         float64
dtypes: float64(4), object(4)
memory usage: 549.5+ MB


,period_begin,region,city,state,median_sale_price,median_list_price,homes_sold,median_dom
0,2020-08-01,"Navarre Beach, FL - Pensacola Beach",Navarre Beach,Florida,467500.0,614500.0,64.0,87.0
1,2020-09-01,"Bayonet Point, FL - Forest Acres",Bayonet Point,Florida,480000.0,499900.0,1.0,88.0
2,2014-02-01,"Scottsdale, AZ - Troon North",Scottsdale,Arizona,543562.0,510000.0,31.0,138.0
3,2019-12-01,"Phoenix, AZ - Bellair",Phoenix,Arizona,185000.0,185000.0,7.0,42.0
4,2016-08-01,"Rochester, MN - Cimarron",Rochester,Minnesota,180000.0,NaN,1.0,88.0


In [145]:
housing_data.isna().sum()
housing_data = housing_data.dropna()
housing_data.shape

(8079016, 8)

In [146]:
housing_data['period_begin'] = pd.to_datetime(housing_data['period_begin'])
housing_data.describe()

,period_begin,median_sale_price,median_list_price,homes_sold,median_dom
count,8079016,8.079016e+06,8.079016e+06,8.079016e+06,8.079016e+06
mean,2017-02-11 22:03:46.756324608,3.847339e+05,4.065877e+05,1.682956e+01,7.222800e+01
min,2012-01-01 00:00:00,1.000000e+00,1.000000e+02,1.000000e+00,1.000000e+00
25%,2014-10-01 00:00:00,1.500000e+05,1.594500e+05,2.000000e+00,2.100000e+01
50%,2017-03-01 00:00:00,2.600000e+05,2.740000e+05,6.000000e+00,4.500000e+01
75%,2019-07-01 00:00:00,4.460000e+05,4.674000e+05,1.400000e+01,8.100000e+01
max,2021-10-01 00:00:00,2.395700e+08,1.000000e+09,3.589000e+03,4.155400e+04
std,NaN,5.105901e+05,8.793095e+05,5.678454e+01,1.594963e+02


In [147]:
# Dictionary of average inflation rates per year
inflation_rates = {
    2012: 1.0207, 2013: 1.0146, 2014: 1.0162, 2015: 1.0012,
    2016: 1.0126, 2017: 1.0213, 2018: 1.0244, 2019: 1.0181,
    2020: 1.0123, 2021: 1.047, 2022: 1.08, 2023: 1.0412, 2024: 1.03
}

# Function to calculate adjusted price to 2024 values
def adjust_to_2024(row):
    year = row['period_begin'].year
    adjusted_price = row['median_sale_price']
    for yr in range(year, 2024):
        adjusted_price *= inflation_rates[yr]
    return adjusted_price

housing_data['sale_price_2024'] = housing_data.apply(adjust_to_2024, axis=1)
housing_data.info()
housing_data.head() # prices adjusted for inflation to 2024

<class 'pandas.core.frame.DataFrame'>
Index: 8079016 entries, 0 to 9002879
Data columns (total 9 columns):
 #   Column             Dtype         
---  ------             -----         
 0   period_begin       datetime64[ns]
 1   region             object        
 2   city               object        
 3   state              object        
 4   median_sale_price  float64       
 5   median_list_price  float64       
 6   homes_sold         float64       
 7   median_dom         float64       
 8   sale_price_2024    float64       
dtypes: datetime64[ns](1), float64(5), object(3)
memory usage: 616.4+ MB


,period_begin,region,city,state,median_sale_price,median_list_price,homes_sold,median_dom,sale_price_2024
0,2020-08-01,"Navarre Beach, FL - Pensacola Beach",Navarre Beach,Florida,467500.0,614500.0,64.0,87.0,557179.909741
1,2020-09-01,"Bayonet Point, FL - Forest Acres",Bayonet Point,Florida,480000.0,499900.0,1.0,88.0,572077.768290
2,2014-02-01,"Scottsdale, AZ - Troon North",Scottsdale,Arizona,543562.0,510000.0,31.0,138.0,710909.331850
3,2019-12-01,"Phoenix, AZ - Bellair",Phoenix,Arizona,185000.0,185000.0,7.0,42.0,224479.144877
5,2017-12-01,"Tucson, AZ - Terra del Sol",Tucson,Arizona,155000.0,157500.0,15.0,43.0,196769.993272


In [150]:
# Extract the neighborhood from the 'region' column
housing_data['neighborhood'] = housing_data['region'].str.split(',', n=1).str[1].str.split('-').str[1].str.strip()

# Drop the 'region' column since it is no longer needed
housing_data = housing_data.drop(columns=['region'])

# Check the updated data
housing_data.info()
housing_data.head()
housing_data.to_csv("Cleaned_Project_Data/housing_data.csv", index=False)

<class 'pandas.core.frame.DataFrame'>
Index: 8079016 entries, 0 to 9002879
Data columns (total 9 columns):
 #   Column             Dtype         
---  ------             -----         
 0   period_begin       datetime64[ns]
 1   city               object        
 2   state              object        
 3   median_sale_price  float64       
 4   median_list_price  float64       
 5   homes_sold         float64       
 6   median_dom         float64       
 7   sale_price_2024    float64       
 8   neighborhood       object        
dtypes: datetime64[ns](1), float64(5), object(3)
memory usage: 616.4+ MB


In [152]:
# Create a new dataset containing the unique neighborhoods
unique_neighborhoods = housing_data[['neighborhood']].drop_duplicates().reset_index(drop=True)

# Check the new dataset
unique_neighborhoods.info()
unique_neighborhoods.head()
unique_neighborhoods.to_csv("Cleaned_Project_Data/unique_neighborhoods.csv", index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27172 entries, 0 to 27171
Data columns (total 1 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   neighborhood  27172 non-null  object
dtypes: object(1)
memory usage: 212.4+ KB


### Data Cleaning Summary

Below is a summary of each step taken to clean the  **Housing Market Data**, the rationale behind it, and the sources that guided the approach.

1. **Column Selection**:  
   I first imported only the most relevant columns to reduce memory usage and focus on variables important for the project. These columns include `period_begin`, `region`, `city`, `state`, `median_sale_price`, `homes_sold`, `median_list_price`, and `median_dom`. This choice minimizes extraneous data and enhances processing speed, particularly with a large dataset containing over 9 million rows.

2. **Handling Missing Data**:  
   The dataset contained some missing values in important columns. After inspecting the data, I found that dropping rows with null values was feasible, as it would only remove around 800,000 rows from a dataset of 9 million rows, retaining over 90% of the data. This approach ensures data integrity without substantial loss.

3. **Datetime Conversion**:  
   The `period_begin` column was converted to a datetime format to enable time-based analysis. This conversion allows for the extraction of the year, which is needed for inflation adjustments, and facilitates future time-based filtering or aggregation.

4. **Inflation Adjustment**:  
   To compare property prices across different years, I adjusted the median sale prices to 2024 values to account for inflation. This adjustment used annual U.S. inflation rates from 2012 to 2024, obtained from reliable sources like [MacroTrends](https://www.macrotrends.net) and [Investopedia](https://www.investopedia.com). By calculating the cumulative inflation effect, the `median_sale_price` was adjusted to reflect equivalent 2024 purchasing power. A new column, `sale_price_2024`, stores these inflation-adjusted prices.

5. **Verification and Inspection**:  
   After applying the adjustments, I verified the data by inspecting the first few rows of the modified columns (`period_begin`, `median_sale_price`, and `sale_price_2024`) to confirm successful transformation and inflation adjustment.

In [89]:
columns_crime = [
    'Record ID', 'Agency Code', 'Agency Name', 'Agency Type', 
    'City', 'State', 'Year', 'Month', 'Incident', 'Crime Type'
]

crime_data = pd.read_csv('US_Crime_DataSet.csv', usecols=columns_crime)

crime_data.info()
print(crime_data.isna().sum())
crime_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 638454 entries, 0 to 638453
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   Record ID    638454 non-null  int64 
 1   Agency Code  638454 non-null  object
 2   Agency Name  638454 non-null  object
 3   Agency Type  638454 non-null  object
 4   City         638454 non-null  object
 5   State        638454 non-null  object
 6   Year         638454 non-null  int64 
 7   Month        638454 non-null  object
 8   Incident     638454 non-null  int64 
 9   Crime Type   638454 non-null  object
dtypes: int64(3), object(7)
memory usage: 48.7+ MB
Record ID      0
Agency Code    0
Agency Name    0
Agency Type    0
City           0
State          0
Year           0
Month          0
Incident       0
Crime Type     0
dtype: int64


,Record ID,Agency Code,Agency Name,Agency Type,City,State,Year,Month,Incident,Crime Type
0,1,AK00101,Anchorage,Municipal Police,Anchorage,Alaska,1980,January,1,Murder or Manslaughter
1,2,AK00101,Anchorage,Municipal Police,Anchorage,Alaska,1980,March,1,Murder or Manslaughter
2,3,AK00101,Anchorage,Municipal Police,Anchorage,Alaska,1980,March,2,Murder or Manslaughter
3,4,AK00101,Anchorage,Municipal Police,Anchorage,Alaska,1980,April,1,Murder or Manslaughter
4,5,AK00101,Anchorage,Municipal Police,Anchorage,Alaska,1980,April,2,Murder or Manslaughter


In [90]:
columns_income = [
    'State_Name', 'City', 'Zip_Code', 'Lat', 'Lon', 'Mean', 'Median' 
]

income_data = pd.read_csv('kaggle_income.csv', usecols=columns_income, encoding='ISO-8859-1')

income_data.info()
print(income_data.isna().sum())
income_data.head()  # likely from 2016, income not adjusted to 2024

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32526 entries, 0 to 32525
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   State_Name  32526 non-null  object 
 1   City        32526 non-null  object 
 2   Zip_Code    32526 non-null  int64  
 3   Lat         32526 non-null  float64
 4   Lon         32526 non-null  float64
 5   Mean        32526 non-null  int64  
 6   Median      32526 non-null  int64  
dtypes: float64(2), int64(3), object(2)
memory usage: 1.7+ MB
State_Name    0
City          0
Zip_Code      0
Lat           0
Lon           0
Mean          0
Median        0
dtype: int64


,State_Name,City,Zip_Code,Lat,Lon,Mean,Median
0,Alabama,Chickasaw,36611,30.771450,-88.079697,38773,30506
1,Alabama,Louisville,36048,31.708516,-85.611039,37725,19528
2,Alabama,Columbiana,35051,33.191452,-86.615618,54606,31930
3,Alabama,Satsuma,36572,30.874343,-88.009442,63919,52814
4,Alabama,Dauphin Island,36528,30.250913,-88.171268,77948,67225


In [117]:
columns_pop = [
    'state_name',
    'city',
    'county_name',
    'zips',
    'lat',
    'lng', 
    'population',
    'density'  
]

pop_data = pd.read_csv('uscities.csv', usecols=columns_pop, encoding='ISO-8859-1')

pop_data.info()
print(pop_data.isna().sum())
pop_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28338 entries, 0 to 28337
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   city         28338 non-null  object 
 1   state_name   28338 non-null  object 
 2   county_name  28338 non-null  object 
 3   lat          28338 non-null  float64
 4   lng          28338 non-null  float64
 5   population   28338 non-null  int64  
 6   density      28338 non-null  int64  
 7   zips         28337 non-null  object 
dtypes: float64(2), int64(2), object(4)
memory usage: 1.7+ MB
city           0
state_name     0
county_name    0
lat            0
lng            0
population     0
density        0
zips           1
dtype: int64


,city,state_name,county_name,lat,lng,population,density,zips
0,New York,New York,New York,40.6943,-73.9249,18713220,10715,11229 11226 11225 11224 11222 11221 11220 1138...
1,Los Angeles,California,Los Angeles,34.1139,-118.4068,12750807,3276,90291 90293 90292 91316 91311 90037 90031 9000...
2,Chicago,Illinois,Cook,41.8373,-87.6862,8604203,4574,60018 60649 60641 60640 60643 60642 60645 6064...
3,Miami,Florida,Miami-Dade,25.7839,-80.2102,6445545,5019,33129 33125 33126 33127 33128 33149 33144 3314...
4,Dallas,Texas,Dallas,32.7936,-96.7662,5743938,1526,75287 75098 75233 75254 75251 75252 75253 7503...


In [118]:
pop_data = pop_data.assign(zips=pop_data['zips'].str.split(' ')).explode('zips')
pop_data.dropna(subset=['zips'])
pop_data.info() # creating row for each zip and unpacking the zips column
pop_data.head(10) # ajusted to current date 2024, last updated 2021

<class 'pandas.core.frame.DataFrame'>
Index: 44956 entries, 0 to 28337
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   city         44956 non-null  object 
 1   state_name   44956 non-null  object 
 2   county_name  44956 non-null  object 
 3   lat          44956 non-null  float64
 4   lng          44956 non-null  float64
 5   population   44956 non-null  int64  
 6   density      44956 non-null  int64  
 7   zips         44955 non-null  object 
dtypes: float64(2), int64(2), object(4)
memory usage: 3.1+ MB


,city,state_name,county_name,lat,lng,population,density,zips
0,New York,New York,New York,40.6943,-73.9249,18713220,10715,11229
0,New York,New York,New York,40.6943,-73.9249,18713220,10715,11226
0,New York,New York,New York,40.6943,-73.9249,18713220,10715,11225
0,New York,New York,New York,40.6943,-73.9249,18713220,10715,11224
0,New York,New York,New York,40.6943,-73.9249,18713220,10715,11222
0,New York,New York,New York,40.6943,-73.9249,18713220,10715,11221
0,New York,New York,New York,40.6943,-73.9249,18713220,10715,11220
0,New York,New York,New York,40.6943,-73.9249,18713220,10715,11385
0,New York,New York,New York,40.6943,-73.9249,18713220,10715,10169
0,New York,New York,New York,40.6943,-73.9249,18713220,10715,10168


In [119]:
pop_data['zips'].astype(float)
pop_data.info()
pop_data.head()
pop_data.to_csv("Cleaned_Project_Data/pop_data.csv", index=False)

<class 'pandas.core.frame.DataFrame'>
Index: 44956 entries, 0 to 28337
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   city         44956 non-null  object 
 1   state_name   44956 non-null  object 
 2   county_name  44956 non-null  object 
 3   lat          44956 non-null  float64
 4   lng          44956 non-null  float64
 5   population   44956 non-null  int64  
 6   density      44956 non-null  int64  
 7   zips         44955 non-null  object 
dtypes: float64(2), int64(2), object(4)
memory usage: 3.1+ MB


In [137]:
columns_living = [
    'state', 'county', 'family_member_count', 
    'housing_cost', 'food_cost', 'healthcare_cost', 
    'childcare_cost', 'taxes', 'total_cost', 
    'median_family_income'
]

living_data = pd.read_csv('cost_of_living_us.csv', usecols=columns_living)

living_data.info()
print(living_data.isna().sum())
living_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31430 entries, 0 to 31429
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   state                 31430 non-null  object 
 1   county                31430 non-null  object 
 2   family_member_count   31430 non-null  object 
 3   housing_cost          31430 non-null  float64
 4   food_cost             31430 non-null  float64
 5   healthcare_cost       31430 non-null  float64
 6   childcare_cost        31430 non-null  float64
 7   taxes                 31430 non-null  float64
 8   total_cost            31430 non-null  float64
 9   median_family_income  31420 non-null  float64
dtypes: float64(7), object(3)
memory usage: 2.4+ MB
state                    0
county                   0
family_member_count      0
housing_cost             0
food_cost                0
healthcare_cost          0
childcare_cost           0
taxes                    0
total_cost 

,state,county,family_member_count,housing_cost,food_cost,healthcare_cost,childcare_cost,taxes,total_cost,median_family_income
0,AL,Autauga County,1p0c,8505.72876,3454.91712,5737.47984,0.0000,6392.94504,39254.0532,73010.414062
1,AL,Autauga County,1p1c,12067.50240,5091.70788,8659.55640,6147.8298,7422.07836,57194.3256,73010.414062
2,AL,Autauga County,1p2c,12067.50240,7460.20308,11581.63260,15824.6940,9769.56228,76141.0308,73010.414062
3,AL,Autauga County,1p3c,15257.15040,9952.23924,14503.70760,18802.1892,13101.70320,94203.5328,73010.414062
4,AL,Autauga County,1p4c,15257.15040,12182.21400,17425.78560,18802.1892,13469.21880,100823.5200,73010.414062


In [138]:
state_mapping = {
    'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas',
    'CA': 'California', 'CO': 'Colorado', 'CT': 'Connecticut', 'DE': 'Delaware',
    'FL': 'Florida', 'GA': 'Georgia', 'HI': 'Hawaii', 'ID': 'Idaho',
    'IL': 'Illinois', 'IN': 'Indiana', 'IA': 'Iowa', 'KS': 'Kansas',
    'KY': 'Kentucky', 'LA': 'Louisiana', 'ME': 'Maine', 'MD': 'Maryland',
    'MA': 'Massachusetts', 'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi',
    'MO': 'Missouri', 'MT': 'Montana', 'NE': 'Nebraska', 'NV': 'Nevada',
    'NH': 'New Hampshire', 'NJ': 'New Jersey', 'NM': 'New Mexico', 'NY': 'New York',
    'NC': 'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio', 'OK': 'Oklahoma',
    'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina',
    'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah',
    'VT': 'Vermont', 'VA': 'Virginia', 'WA': 'Washington', 'WV': 'West Virginia',
    'WI': 'Wisconsin', 'WY': 'Wyoming',
    'PR': 'Puerto Rico', 'GU': 'Guam', 'VI': 'Virgin Islands',
    'FM': 'Federated States of Micronesia', 'AS': 'American Samoa', 'MP': 'Northern Mariana Islands', 'MH': 'Marshall Islands'
}

living_data['state_name'] = living_data['state'].map(state_mapping)

living_data.head()

,state,county,family_member_count,housing_cost,food_cost,healthcare_cost,childcare_cost,taxes,total_cost,median_family_income,state_name
0,AL,Autauga County,1p0c,8505.72876,3454.91712,5737.47984,0.0000,6392.94504,39254.0532,73010.414062,Alabama
1,AL,Autauga County,1p1c,12067.50240,5091.70788,8659.55640,6147.8298,7422.07836,57194.3256,73010.414062,Alabama
2,AL,Autauga County,1p2c,12067.50240,7460.20308,11581.63260,15824.6940,9769.56228,76141.0308,73010.414062,Alabama
3,AL,Autauga County,1p3c,15257.15040,9952.23924,14503.70760,18802.1892,13101.70320,94203.5328,73010.414062,Alabama
4,AL,Autauga County,1p4c,15257.15040,12182.21400,17425.78560,18802.1892,13469.21880,100823.5200,73010.414062,Alabama


In [139]:
def extract_family_count(family_count_str):
    parent_count, child_count = family_count_str.split('p')
    child_count = child_count.split('c')[0]  # Extract the number before 'c'
    return int(parent_count) + int(child_count)

living_data['family_member_count_int'] = living_data['family_member_count'].apply(extract_family_count)

living_data[['family_member_count', 'family_member_count_int']].head(10)

,family_member_count,family_member_count_int
0,1p0c,1
1,1p1c,2
2,1p2c,3
3,1p3c,4
4,1p4c,5
5,2p0c,2
6,2p1c,3
7,2p2c,4
8,2p3c,5
9,2p4c,6


In [140]:
living_data = living_data.drop(columns=['family_member_count'])

living_data = living_data.rename(columns={'family_member_count_int': 'family_member_count'})

living_data.dropna(subset=["median_family_income"])
living_data.head()
living_data.to_csv("Cleaned_Project_Data/living_data.csv", index=False)

In [141]:
columns_jobs = [
    'AREA_TITLE', 'PRIM_STATE', 'TOT_EMP', 'JOBS_1000',
    'H_MEAN', 'A_MEAN', 'H_PCT10', 'H_PCT25', 'H_MEDIAN',
    'H_PCT75', 'H_PCT90', 'A_PCT10', 'A_PCT25', 'A_MEDIAN',
    'A_PCT75', 'A_PCT90'
]

# Load the data using the specified columns
jobs_data = pd.read_csv('MSA_M2023_dl.csv', usecols=columns_jobs)

# Clean the 'AREA_TITLE' column
# Remove everything after the first comma
jobs_data['AREA_TITLE'] = jobs_data['AREA_TITLE'].str.split(',', n=1).str[0]

# Split cities separated by dashes into individual entries (if needed)
# Example: 'City1-City2' becomes two separate rows
jobs_data = jobs_data.assign(AREA_TITLE=jobs_data['AREA_TITLE'].str.split('-')).explode('AREA_TITLE')

# Strip any leading/trailing spaces from the 'AREA_TITLE' column
jobs_data['AREA_TITLE'] = jobs_data['AREA_TITLE'].str.strip()

# Add a new column with the full state names
jobs_data['PRIM_STATE'] = jobs_data['PRIM_STATE'].str.upper()
jobs_data['state_name'] = jobs_data['PRIM_STATE'].map(state_mapping)

jobs_data = jobs_data.rename(columns={'AREA_TITLE': 'city'})

# Check the cleaned data
jobs_data.info()
print(jobs_data.isna().sum())
jobs_data.head(10)

<class 'pandas.core.frame.DataFrame'>
Index: 253226 entries, 0 to 149590
Data columns (total 17 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   city        253226 non-null  object
 1   PRIM_STATE  253226 non-null  object
 2   TOT_EMP     253226 non-null  object
 3   JOBS_1000   253226 non-null  object
 4   H_MEAN      253226 non-null  object
 5   A_MEAN      253226 non-null  object
 6   H_PCT10     253226 non-null  object
 7   H_PCT25     253226 non-null  object
 8   H_MEDIAN    253226 non-null  object
 9   H_PCT75     253226 non-null  object
 10  H_PCT90     253226 non-null  object
 11  A_PCT10     253226 non-null  object
 12  A_PCT25     253226 non-null  object
 13  A_MEDIAN    253226 non-null  object
 14  A_PCT75     253226 non-null  object
 15  A_PCT90     253226 non-null  object
 16  state_name  250976 non-null  object
dtypes: object(17)
memory usage: 34.8+ MB
city             0
PRIM_STATE       0
TOT_EMP          0
JOBS_1000       

,city,PRIM_STATE,TOT_EMP,JOBS_1000,H_MEAN,A_MEAN,H_PCT10,H_PCT25,H_MEDIAN,H_PCT75,H_PCT90,A_PCT10,A_PCT25,A_MEDIAN,A_PCT75,A_PCT90,state_name
0,Abilene,TX,"72,780",1000.000,23.93,"49,780",10.94,13.80,18.60,28.03,40.08,"22,750","28,710","38,680","58,310","83,360",Texas
1,Abilene,TX,"4,930",67.752,48.41,"100,690",20.23,28.50,39.09,58.27,81.79,"42,080","59,290","81,310","121,200","170,130",Texas
2,Abilene,TX,40,0.561,115.39,"240,020",47.35,65.04,97.28,#,#,"98,490","135,280","202,340",#,#,Texas
3,Abilene,TX,"2,290",31.404,47.35,"98,480",16.56,23.77,37.19,57.85,86.97,"34,440","49,430","77,360","120,330","180,900",Texas
4,Abilene,TX,120,1.675,55.91,"116,290",28.91,36.05,45.31,72.66,97.35,"60,130","74,980","94,240","151,130","202,490",Texas
5,Abilene,TX,250,3.433,51.83,"107,810",22.54,29.21,44.89,62.57,87.76,"46,870","60,760","93,370","130,140","182,530",Texas
6,Abilene,TX,30,0.420,42.76,"88,930",26.44,30.62,35.61,46.71,62.93,"55,000","63,680","74,070","97,160","130,900",Texas
7,Abilene,TX,80,1.117,57.65,"119,910",35.41,39.85,46.03,60.83,76.67,"73,650","82,890","95,750","126,520","159,470",Texas
8,Abilene,TX,30,0.448,42.41,"88,200",27.74,30.71,37.82,48.31,63.42,"57,690","63,890","78,670","100,480","131,920",Texas
9,Abilene,TX,120,1.671,66.29,"137,880",32.67,47.96,63.29,79.70,98.19,"67,940","99,750","131,640","165,770","204,220",Texas


In [142]:
# Columns that start with 'H' are hourly wage columns, and those that start with 'A' are annual wage columns
hourly_columns = [col for col in jobs_data.columns if col.startswith('H')]
annual_columns = [col for col in jobs_data.columns if col.startswith('A')]

# Replace "#" in hourly columns with 150 and in annual columns with 250000
for col in hourly_columns:
    jobs_data[col] = jobs_data[col].astype(str).replace('#', '150')

for col in annual_columns:
    jobs_data[col] = jobs_data[col].astype(str).replace('#', '250000')

# Remove commas from strings, replace "*" and "**" with NaN, and convert to float
for col in hourly_columns + annual_columns + ["TOT_EMP", "JOBS_1000"]:
    jobs_data[col] = (
        jobs_data[col]
        .str.replace(',', '', regex=False)  # Remove commas
        .replace(r'[\*\*?]', np.nan, regex=True)  # Replace * and ** with NaN
        .astype(float)  # Convert to float
    )

# Drop rows where 'state_name' is NaN, if necessary
jobs_data = jobs_data.dropna(subset=["state_name"])

# Check the cleaned data
jobs_data.info()
jobs_data.head()
jobs_data.to_csv("Cleaned_Project_Data/jobs_data.csv", index=False)

<class 'pandas.core.frame.DataFrame'>
Index: 250976 entries, 0 to 149590
Data columns (total 17 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   city        250976 non-null  object 
 1   PRIM_STATE  250976 non-null  object 
 2   TOT_EMP     246978 non-null  float64
 3   JOBS_1000   246978 non-null  float64
 4   H_MEAN      234851 non-null  float64
 5   A_MEAN      248302 non-null  float64
 6   H_PCT10     234851 non-null  float64
 7   H_PCT25     234851 non-null  float64
 8   H_MEDIAN    234851 non-null  float64
 9   H_PCT75     234851 non-null  float64
 10  H_PCT90     234851 non-null  float64
 11  A_PCT10     248302 non-null  float64
 12  A_PCT25     248302 non-null  float64
 13  A_MEDIAN    248302 non-null  float64
 14  A_PCT75     248302 non-null  float64
 15  A_PCT90     248302 non-null  float64
 16  state_name  250976 non-null  object 
dtypes: float64(14), object(3)
memory usage: 34.5+ MB


In [99]:
housing_data.to_csv("Cleaned_Project_Data/housing_data.csv", index=False)
crime_data.to_csv("Cleaned_Project_Data/crime_data.csv", index=False)
income_data.to_csv("Cleaned_Project_Data/income_data.csv", index=False)
pop_data.to_csv("Cleaned_Project_Data/pop_data.csv", index=False)
living_data.to_csv("Cleaned_Project_Data/living_data.csv", index=False)
jobs_data.to_csv("Cleaned_Project_Data/jobs_data.csv", index=False)

In [143]:
# List of columns to include
columns_climate = [
    'disaster_number', 'state', 'declaration_date', 'declaration_type',
    'incident_type', 'ih_program_declared', 'ia_program_declared',
    'pa_program_declared', 'hm_program_declared'
]

# Load the data using the specified columns
climate_data = pd.read_csv('us_disaster_declarations.csv', usecols=columns_climate)

# Convert 'declaration_date' to datetime
climate_data['declaration_date'] = pd.to_datetime(climate_data['declaration_date']).dt.strftime('%Y-%m-%d')

# Create a new column 'programs_declared_count' that sums the binary flags
climate_data['programs_declared_count'] = (
    climate_data['ih_program_declared'] +
    climate_data['ia_program_declared'] +
    climate_data['pa_program_declared'] +
    climate_data['hm_program_declared']
)

climate_data = climate_data.drop(columns=['ih_program_declared', 'ia_program_declared',
    'pa_program_declared', 'hm_program_declared'])
climate_data['state_name'] = climate_data['state'].str.upper().map(state_mapping)
# Check the cleaned data
climate_data.info()
print(climate_data.isna().sum())
climate_data.head(10)
climate_data.to_csv("Cleaned_Project_Data/climate_data.csv", index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64092 entries, 0 to 64091
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   disaster_number          64092 non-null  int64 
 1   state                    64092 non-null  object
 2   declaration_type         64092 non-null  object
 3   declaration_date         64092 non-null  object
 4   incident_type            64092 non-null  object
 5   programs_declared_count  64092 non-null  int64 
 6   state_name               64068 non-null  object
dtypes: int64(2), object(5)
memory usage: 3.4+ MB
disaster_number             0
state                       0
declaration_type            0
declaration_date            0
incident_type               0
programs_declared_count     0
state_name                 24
dtype: int64


In [136]:
climate_data[climate_data['state_name'].isna()]['state'].head(10)

4904    MP
6724    MP
7457    MP
8758    MP
8803    MH
8858    MP
8859    MP
8860    MP
8936    MH
8937    MP
Name: state, dtype: object

In [180]:
# Cleaning County
import pandas as pd

column_names = ['county', 'state_id', 'lat', 'lng', 'population', 'density', 'family_member_count', 
                'housing_cost', 'food_cost', 'healthcare_cost', 'childcare_cost', 'taxes', 'total_cost', 
                'median_family_income']
county_data = pd.read_csv('county.csv', header=None, names=column_names)
county_data.info()
county_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86652 entries, 0 to 86651
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   county                86652 non-null  object 
 1   state_id              86652 non-null  object 
 2   lat                   86652 non-null  float64
 3   lng                   86652 non-null  float64
 4   population            86652 non-null  int64  
 5   density               86652 non-null  int64  
 6   family_member_count   86652 non-null  int64  
 7   housing_cost          86652 non-null  float64
 8   food_cost             86652 non-null  float64
 9   healthcare_cost       86652 non-null  float64
 10  childcare_cost        86652 non-null  float64
 11  taxes                 86652 non-null  float64
 12  total_cost            86652 non-null  float64
 13  median_family_income  86652 non-null  float64
dtypes: float64(9), int64(3), object(2)
memory usage: 9.3+ MB


,county,state_id,lat,lng,population,density,family_member_count,housing_cost,food_cost,healthcare_cost,childcare_cost,taxes,total_cost,median_family_income
0,Los Angeles County,CA,34.3219,-118.2247,9936690,2421,1,16428.0,3589.7933,3884.3800,0.0000,7876.4882,48890.8596,78729.8359
1,Los Angeles County,CA,34.3219,-118.2247,9936690,2421,2,21978.0,5935.8858,6800.4388,5181.8324,9778.9182,71077.5876,78729.8359
2,Los Angeles County,CA,34.3219,-118.2247,9936690,2421,3,24696.0,7971.2589,8748.1765,13121.7506,12166.0433,90916.8924,78729.8359
3,Los Angeles County,CA,34.3219,-118.2247,9936690,2421,4,28758.0,10357.2931,10695.9137,16728.4788,16218.2856,110689.0254,78729.8359
4,Los Angeles County,CA,34.3219,-118.2247,9936690,2421,5,32820.0,12661.7214,12643.6519,17577.1212,19027.0266,125589.6720,78729.8359


In [181]:
unique_county_data = county_data.drop_duplicates(
    subset=['county', 'state_id', 'family_member_count'],
    keep='first'  # Keeps the first occurrence and drops the rest
).reset_index(drop=True)  # Optional: Reset index after dropping duplicates

# Display the cleaned DataFrame
print("\nData After Removing Duplicates:")
unique_county_data.head()
unique_county_data.info()


Data After Removing Duplicates:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18378 entries, 0 to 18377
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   county                18378 non-null  object 
 1   state_id              18378 non-null  object 
 2   lat                   18378 non-null  float64
 3   lng                   18378 non-null  float64
 4   population            18378 non-null  int64  
 5   density               18378 non-null  int64  
 6   family_member_count   18378 non-null  int64  
 7   housing_cost          18378 non-null  float64
 8   food_cost             18378 non-null  float64
 9   healthcare_cost       18378 non-null  float64
 10  childcare_cost        18378 non-null  float64
 11  taxes                 18378 non-null  float64
 12  total_cost            18378 non-null  float64
 13  median_family_income  18378 non-null  float64
dtypes: float64(9), int64(3), object(2)
me

In [183]:
numeric_cols = ['lat', 'lng', 'population', 'density', 
           'housing_cost', 'food_cost', 'healthcare_cost', 'childcare_cost',
             'taxes', 'total_cost', 'median_family_income']

# Define the columns to group by
group_cols = ['county', 'state_id', 'family_member_count']

agg_functions = {col: 'mean' for col in numeric_cols}

county_data_aggregated = county_data.groupby(group_cols).agg(agg_functions).reset_index()

county_data_aggregated.head()
county_data_aggregated.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18378 entries, 0 to 18377
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   county                18378 non-null  object 
 1   state_id              18378 non-null  object 
 2   family_member_count   18378 non-null  int64  
 3   lat                   18378 non-null  float64
 4   lng                   18378 non-null  float64
 5   population            18378 non-null  float64
 6   density               18378 non-null  float64
 7   housing_cost          18378 non-null  float64
 8   food_cost             18378 non-null  float64
 9   healthcare_cost       18378 non-null  float64
 10  childcare_cost        18378 non-null  float64
 11  taxes                 18378 non-null  float64
 12  total_cost            18378 non-null  float64
 13  median_family_income  18378 non-null  float64
dtypes: float64(11), int64(1), object(2)
memory usage: 2.0+ MB


In [177]:
duplicate_counts = county_data.groupby(['county', 'state_id', 'family_member_count']).size().reset_index(name='count')

duplicates = duplicate_counts[duplicate_counts['count'] > 1]

duplicates.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10092 entries, 24 to 18359
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   county               10092 non-null  object
 1   state_id             10092 non-null  object
 2   family_member_count  10092 non-null  int64 
 3   count                10092 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 394.2+ KB


In [184]:
county_data_aggregated.to_csv("Cleaned_Project_Data/county_data_aggregated.csv", index=False)

In [197]:
# loading city_to_zip_code.csv
column_names1 = ['city', 'state_id', 'zip_code']
city_to_zip_code_data = pd.read_csv('city_to_zip_code.csv', header=None, names=column_names1)
city_to_zip_code_data.info()
city_to_zip_code_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43896 entries, 0 to 43895
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   city      43896 non-null  object
 1   state_id  43896 non-null  object
 2   zip_code  43896 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


,city,state_id,zip_code
0,New York,NY,11229
1,New York,NY,11226
2,New York,NY,11225
3,New York,NY,11224
4,New York,NY,11222


In [198]:
column_names2 = ['code', 'median_income', 'city', 'state_id']
zip_code_data = pd.read_csv('zip_code.csv', header=None, names=column_names2)
zip_code_data.info()
zip_code_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12734 entries, 0 to 12733
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   code           12734 non-null  int64  
 1   median_income  12734 non-null  float64
 2   city           12734 non-null  object 
 3   state_id       12734 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 398.1+ KB


,code,median_income,city,state_id
0,36867,79218.0,Ozark,AL
1,93643,25394.0,North Fork,CA
2,6441,115269.0,Higganum,CT
3,70665,37725.0,Sulphur,LA
4,33180,93220.0,Miami,FL


In [199]:
filtered_data = city_to_zip_code_data[~city_to_zip_code_data['zip_code'].isin(zip_code_data['code'])]

extra_zip_code = filtered_data.drop_duplicates(subset='zip_code').reset_index(drop=True)

# Display the result
print(extra_zip_code)

               city state_id  zip_code
0          New York       NY     11385
1          New York       NY     10169
2          New York       NY     10168
3          New York       NY     10167
4          New York       NY     10165
...             ...      ...       ...
19190          Ruso       ND     58778
19191    Elfin Cove       AK     99825
19192  Prairie City       SD     57649
19193        Lowman       ID     83637
19194      Shamrock       OK     74068

[19195 rows x 3 columns]


In [200]:
# Step 4: Rename 'zip_code' to 'code' to match zip_code_data
extra_zip_code = extra_zip_code.rename(columns={'zip_code': 'code'})

# Step 5: Add 'median_income' column with NaN
extra_zip_code['median_income'] = np.nan

# Step 6: Reorder columns to match zip_code_data
extra_zip_code = extra_zip_code[['code', 'median_income', 'city', 'state_id']]

# Step 7: Concatenate zip_code_data with extra_zip_code
combined_zip_code_data = pd.concat([zip_code_data, extra_zip_code], ignore_index=True)
combined_zip_code_data.info()
combined_zip_code_data.to_csv("Cleaned_Project_Data/combined_zip_code_data.csv", index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31929 entries, 0 to 31928
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   code           31929 non-null  int64  
 1   median_income  12734 non-null  float64
 2   city           31929 non-null  object 
 3   state_id       31929 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 997.9+ KB


In [204]:
import pandas as pd

zip_county_columns = ['Zip Code', 'County Name']

zip_county_data = pd.read_csv('us_zip_fips_county.csv', encoding='ISO-8859-1', usecols=zip_county_columns)

zip_county_data.info()
zip_county_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41867 entries, 0 to 41866
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Zip Code     41867 non-null  int64 
 1   County Name  41867 non-null  object
dtypes: int64(1), object(1)
memory usage: 654.3+ KB


,Zip Code,County Name
0,501,Suffolk County
1,544,Suffolk County
2,6390,Suffolk County
3,11701,Suffolk County
4,11702,Suffolk County


In [205]:
column_names2 = ['code', 'median_income', 'city', 'state_id']
zip_code_data = pd.read_csv('zip_code.csv', header=None, names=column_names2)
zip_code_data.info()
zip_code_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31929 entries, 0 to 31928
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   code           31929 non-null  int64  
 1   median_income  12734 non-null  float64
 2   city           31929 non-null  object 
 3   state_id       31929 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 997.9+ KB


,code,median_income,city,state_id
0,36867,79218.0,Ozark,AL
1,93643,25394.0,North Fork,CA
2,6441,115269.0,Higganum,CT
3,70665,37725.0,Sulphur,LA
4,33180,93220.0,Miami,FL


In [206]:
merged_zip_county_data = pd.merge(
    zip_code_data,
    zip_county_data,
    how='left',
    left_on='code',
    right_on='Zip Code'
)

# Drop the redundant 'Zip Code' column from the first dataset
merged_zip_county_data.drop('Zip Code', axis=1, inplace=True)

# Optionally, rename 'County Name' for clarity
merged_zip_county_data.rename(columns={'County Name': 'county'}, inplace=True)

merged_zip_county_data.info()
merged_zip_county_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31930 entries, 0 to 31929
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   code           31930 non-null  int64  
 1   median_income  12734 non-null  float64
 2   city           31930 non-null  object 
 3   state_id       31930 non-null  object 
 4   county         31833 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 1.2+ MB


,code,median_income,city,state_id,county
0,36867,79218.0,Ozark,AL,Russell County
1,93643,25394.0,North Fork,CA,Madera County
2,6441,115269.0,Higganum,CT,Middlesex County
3,70665,37725.0,Sulphur,LA,Calcasieu Parish
4,33180,93220.0,Miami,FL,Miami-Dade County


In [210]:
# Check how many missing counties there are
missing_counties = merged_zip_county_data['county'].isnull().sum()
print(f"Number of zip codes without a county match: {missing_counties}")

# Optionally, fill missing counties with a placeholder
merged_zip_county_data['county'] = merged_zip_county_data['county'].fillna('Unknown')
merged_zip_county_data.head()

Number of zip codes without a county match: 0


,code,median_income,city,state_id,county
0,36867,79218.0,Ozark,AL,Russell County
1,93643,25394.0,North Fork,CA,Madera County
2,6441,115269.0,Higganum,CT,Middlesex County
3,70665,37725.0,Sulphur,LA,Calcasieu Parish
4,33180,93220.0,Miami,FL,Miami-Dade County


In [211]:
merged_zip_county_data.to_csv("Cleaned_Project_Data/merged_zip_county_data.csv", index=False)

In [214]:
zip_code_data[zip_code_data['median_income'] == 0]['code'].head(20)

161     35178
602     99901
615     99588
697     86441
738     85334
783     85333
895     31305
924     48617
942     85034
1247     6114
1320    92840
1841    93204
1925    70510
2083    19968
2375    93405
2402    70615
2424    32164
2485    33172
2498    95127
3170    32625
Name: code, dtype: int64